In [4]:
import pandas as pd
import numpy as np

df_base = pd.read_csv("/home/c-crchen/swdump202/ParamISP/ParamISP/runs/metrics/param-isp/metrics.csv")
df = pd.read_csv("/home/c-crchen/swdump202/ParamISP/ParamISP/runs/metrics/fivek/metrics.csv")

log_scale = ["exposure_time", "iso"]
for col in log_scale:
    df_base[col] = np.log10(df_base[col])
    df[col] = np.log10(df[col])


In [18]:
x_axis = "exposure_time"
y_axis = "iso"

# Define thresholds for x and y axes (in log10 space)
x_thresh = 0.2  # e.g., within 0.1 log10 units for iso
y_thresh = 0.2  # e.g., within 0.1 log10 units for exposure_time

# Prepare to store the averaged metrics for each row in df
avg_psnr = []
avg_ssim = []
avg_lpips = []

# Prepare to store the averaged metrics for each row in df
avg_psnr = []
avg_ssim = []
avg_lpips = []
has_neighbors = []

for idx, row in df.iterrows():
    x_val = row[x_axis]
    y_val = row[y_axis]
    # Find df_base rows within the threshold distance in both axes
    mask = (
        (np.abs(df_base[x_axis] - x_val) < x_thresh) &
        (np.abs(df_base[y_axis] - y_val) < y_thresh)
    )
    neighbors = df_base[mask]
    if not neighbors.empty:
        avg_psnr.append(neighbors["psnr"].mean())
        avg_ssim.append(neighbors["ssim"].mean())
        avg_lpips.append(neighbors["lpips"].mean())
        has_neighbors.append(True)
    else:
        avg_psnr.append(np.nan)
        avg_ssim.append(np.nan)
        avg_lpips.append(np.nan)
        has_neighbors.append(False)

# Add the averaged metrics and neighbor flag as new columns to df
df["base_avg_psnr"] = avg_psnr
df["base_avg_ssim"] = avg_ssim
df["base_avg_lpips"] = avg_lpips
df["has_neighbors"] = has_neighbors

# Split into two DataFrames: one with neighbors, one without
df_with_neighbors = df[df["has_neighbors"]].copy()
df_without_neighbors = df[~df["has_neighbors"]].copy()


In [24]:
compare_df = pd.DataFrame({
    "with_neighbors_mean": [
        len(df_with_neighbors),
        df_with_neighbors["psnr"].mean(),
        df_with_neighbors["ssim"].mean(),
        df_with_neighbors["lpips"].mean(),
        df_with_neighbors["base_avg_psnr"].mean(),
        df_with_neighbors["base_avg_ssim"].mean(),
        df_with_neighbors["base_avg_lpips"].mean(),
    ],
    "without_neighbors_mean": [
        len(df_without_neighbors),
        df_without_neighbors["psnr"].mean(),
        df_without_neighbors["ssim"].mean(),
        df_without_neighbors["lpips"].mean(),
        df_without_neighbors["base_avg_psnr"].mean(),
        df_without_neighbors["base_avg_ssim"].mean(),
        df_without_neighbors["base_avg_lpips"].mean(),
    ]
}, index=pd.Index([
    "#samples",
    "psnr", 
    "ssim", 
    "lpips", 
    "base_avg_psnr",
    "base_avg_ssim",
    "base_avg_lpips",
]))

compare_df

,with_neighbors_mean,without_neighbors_mean
#samples,3070.000000,1555.000000
psnr,19.151221,19.709192
ssim,0.740429,0.712294
lpips,0.329964,0.373765
base_avg_psnr,29.993578,NaN
base_avg_ssim,0.914427,NaN
base_avg_lpips,0.112195,NaN
